**After Processing the raw data in the end to end pipeline matlab file, this python notebook is meant to segment and form them into .npy files which are easily lodaded into our custom deep learning model**

A. Segment for Train set

In [ ]:
import os
import numpy as np
from glob import glob
import scipy.io as sio

# Parameters for CWT segmentation based on time resolution
segment_duration = 2.5    # seconds
fs = 800                  # sampling frequency (Hz)
segment_size = int(fs * segment_duration)  # 800*2.5 = 2000 frames
overlap = 0.75
step_size = int(segment_size * (1 - overlap))

# The input and output directories for the train CWT data (can adjust the paths as needed)
input_path = "/content/drive/MyDrive/my_dataset/traincwt/" # Change to suitable path
output_path = "/content/drive/MyDrive/my_dataset/traincwt_segmented/" # Change to suitable path
os.makedirs(output_path, exist_ok=True)

# Get list of all CWT amplitude files for the train set
amp_files = sorted(glob(os.path.join(input_path, "trainamp_*_cwt.mat")))

for amp_file in amp_files:
    amp_filename = os.path.basename(amp_file)

    # Find corresponding phase file by replacing "trainamp_" with "trainphase_"
    phase_filename = amp_filename.replace("trainamp_", "trainphase_")
    phase_file = os.path.join(input_path, phase_filename)

    if not os.path.exists(phase_file):
        print(f"Skipping {amp_filename} as phase file {phase_filename} not found.")
        continue

    # Load CWT amplitude and phase data from .mat files
    amp_mat = sio.loadmat(amp_file)
    phase_mat = sio.loadmat(phase_file)

    # Check for required variables
    if "wt_amp" not in amp_mat or "wt_phase" not in phase_mat:
        print(f"Variables not found in {amp_filename} or {phase_filename}, skipping.")
        continue

    amp_data = amp_mat["wt_amp"]   # Expected shape: (freq_bins, time_frames)
    phase_data = phase_mat["wt_phase"]

    # Checking the dimensions
    if amp_data.shape != phase_data.shape:
        print(f"Mismatch in dimensions for {amp_filename} and {phase_filename}, skipping.")
        continue

    freq_bins, total_frames = amp_data.shape
    segment_num = 1
    start_idx = 0

    # Segment along the time axis (columns)
    while start_idx + segment_size <= total_frames:
        amp_segment = amp_data[:, start_idx:start_idx + segment_size]
        phase_segment = phase_data[:, start_idx:start_idx + segment_size]

        # Create output filenames (.npy format)
        base_name = amp_filename[:-4]  # Removes the '.mat' extension
        amp_segment_name = f"{base_name}_S{segment_num}.npy".replace("trainamp", "segtrainamp")
        phase_segment_name = f"{base_name}_S{segment_num}.npy".replace("trainamp", "segtrainphase")
        amp_segment_file = os.path.join(output_path, amp_segment_name)
        phase_segment_file = os.path.join(output_path, phase_segment_name)

        # Save segments as .npy files
        np.save(amp_segment_file, amp_segment)
        np.save(phase_segment_file, phase_segment)

        start_idx += step_size
        segment_num += 1

    # Handle any leftover frames as a final segment
    if total_frames - start_idx > 0:
        remaining = total_frames - start_idx
        extra_needed = segment_size - remaining
        extra_start = max(0, start_idx - extra_needed)

        amp_segment = amp_data[:, extra_start:]
        phase_segment = phase_data[:, extra_start:]

        base_name = amp_filename[:-4]
        amp_segment_name = f"{base_name}_S{segment_num}.npy".replace("trainamp", "segtrainamp")
        phase_segment_name = f"{base_name}_S{segment_num}.npy".replace("trainamp", "segtrainphase")
        amp_segment_file = os.path.join(output_path, amp_segment_name)
        phase_segment_file = os.path.join(output_path, phase_segment_name)

        np.save(amp_segment_file, amp_segment)
        np.save(phase_segment_file, phase_segment)

    print(f"Segmented CWT data for {amp_filename} and its phase file.")

print("All CWT segmentation tasks for the train set completed.")


B. Segment for test set

In [ ]:
import os
import numpy as np
from glob import glob
import scipy.io as sio

# Parameters for CWT segmentation based on time resolution
segment_duration = 2.5    # seconds
fs = 800                  # sampling frequency (Hz)
segment_size = int(fs * segment_duration)  # 800*2.5 = 2000 frames
overlap = 0.75
step_size = int(segment_size * (1 - overlap))

# The input and output directories for the test CWT data (can adjust the paths as needed)
input_path = "/content/drive/MyDrive/my_dataset/testcwt/" # Change to suitable path
output_path = "/content/drive/MyDrive/my_dataset/testcwt_segmented/" # Change to suitable path
os.makedirs(output_path, exist_ok=True)

# Get list of all CWT amplitude files for the test set
amp_files = sorted(glob(os.path.join(input_path, "testamp_*_cwt.mat")))

for amp_file in amp_files:
    amp_filename = os.path.basename(amp_file)

    # Find corresponding phase file by replacing "testamp_" with "testphase_"
    phase_filename = amp_filename.replace("testamp_", "testphase_")
    phase_file = os.path.join(input_path, phase_filename)

    if not os.path.exists(phase_file):
        print(f"Skipping {amp_filename} as phase file {phase_filename} not found.")
        continue

    # Load CWT amplitude and phase data from .mat files
    amp_mat = sio.loadmat(amp_file)
    phase_mat = sio.loadmat(phase_file)

    # Check for required variables
    if "wt_amp" not in amp_mat or "wt_phase" not in phase_mat:
        print(f"Variables not found in {amp_filename} or {phase_filename}, skipping.")
        continue

    amp_data = amp_mat["wt_amp"]   # Expected shape: (freq_bins, time_frames)
    phase_data = phase_mat["wt_phase"]

    if amp_data.shape != phase_data.shape:
        print(f"Mismatch in dimensions for {amp_filename} and {phase_filename}, skipping.")
        continue

    freq_bins, total_frames = amp_data.shape
    segment_num = 1
    start_idx = 0

    # Segment along the time axis (columns)
    while start_idx + segment_size <= total_frames:
        amp_segment = amp_data[:, start_idx:start_idx + segment_size]
        phase_segment = phase_data[:, start_idx:start_idx + segment_size]

        # Create output filenames (.npy format)
        base_name = amp_filename[:-4]  # Remove '.mat' extension
        amp_segment_name = f"{base_name}_S{segment_num}.npy".replace("testamp", "segtestamp")
        phase_segment_name = f"{base_name}_S{segment_num}.npy".replace("testamp", "segtestphase")
        amp_segment_file = os.path.join(output_path, amp_segment_name)
        phase_segment_file = os.path.join(output_path, phase_segment_name)

        # Save segments as .npy files
        np.save(amp_segment_file, amp_segment)
        np.save(phase_segment_file, phase_segment)

        start_idx += step_size
        segment_num += 1

    # Handle any leftover frames as a final segment
    if total_frames - start_idx > 0:
        remaining = total_frames - start_idx
        extra_needed = segment_size - remaining
        extra_start = max(0, start_idx - extra_needed)

        amp_segment = amp_data[:, extra_start:]
        phase_segment = phase_data[:, extra_start:]

        base_name = amp_filename[:-4]
        amp_segment_name = f"{base_name}_S{segment_num}.npy".replace("testamp", "segtestamp")
        phase_segment_name = f"{base_name}_S{segment_num}.npy".replace("testamp", "segtestphase")
        amp_segment_file = os.path.join(output_path, amp_segment_name)
        phase_segment_file = os.path.join(output_path, phase_segment_name)

        np.save(amp_segment_file, amp_segment)
        np.save(phase_segment_file, phase_segment)

    print(f"Segmented CWT data for {amp_filename} and its phase file.")

print("All CWT segmentation tasks for the test set completed.")


C. Preparing train data

In [ ]:
import os
import numpy as np
from glob import glob
from sklearn.utils import shuffle

segmented_dir = "/content/drive/MyDrive/my_dataset/traincwt_segmented/"
preprocessed_dir = "/content/drive/MyDrive/my_dataset/preprocessedcwt/"
os.makedirs(preprocessed_dir, exist_ok=True)

# Files to save preprocessed training data
train_combined_file = os.path.join(preprocessed_dir, "X_train_combinedcwt.npy")
train_label_file = os.path.join(preprocessed_dir, "Y_traincwt.npy")
global_mean_file = os.path.join(preprocessed_dir, "global_mean.npy")
global_std_file  = os.path.join(preprocessed_dir, "global_std.npy")

# ----- Function to load and combine training data -----
def load_train_data():
    # retrieve all segmented amplitude files
    amp_files = sorted(glob(os.path.join(segmented_dir, "segtrainamp_*.npy")))
    X_combined = []  # store the combined amplitude-phase arrays
    Y = []           # Labels

    for amp_file in amp_files:
        # Derive corresponding phase file name by replacing "segtrainamp_" with "segtrainphase_"
        phase_file = amp_file.replace("segtrainamp_", "segtrainphase_")
        if not os.path.exists(phase_file):
            print(f"Skipping {amp_file}, corresponding phase file not found.")
            continue

        # Load amplitude and phase arrays
        amp_data = np.load(amp_file)      # Expected shape: (freq_bins, time_frames)
        phase_data = np.load(phase_file)  # Expected shape: (freq_bins, time_frames)

        print(f"Loaded amplitude file: {amp_file} with shape {amp_data.shape}")
        print(f"Loaded phase file: {phase_file} with shape {phase_data.shape}")

        if amp_data.shape != phase_data.shape:
            print(f"Mismatch in dimensions for {amp_file}, skipping.")
            continue

        # Stack amplitude and phase along a new last axis -> shape: (freq_bins, time_frames, 2)
        combined = np.stack([amp_data, phase_data], axis=-1)
        X_combined.append(combined)

        # Extract label from the file name.
        # Given "segtrainamp_A1_P0_L1_T2_cwt_S1.npy",
        # we assume the activity label is encoded after "_A" (here, "1") and shift to 0-based indexing.
        try:
            label_str = amp_file.split("_A")[1].split("_")[0]
            label = int(label_str) - 1
        except Exception as e:
            print(f"Error extracting label from {amp_file}: {e}")
            continue

        Y.append(label)

    X_combined = np.array(X_combined)  # Shape: (num_samples, freq_bins, time_frames, 2)
    Y = np.array(Y)
    return X_combined, Y

# ----- Load or Process Training Data -----
if os.path.exists(train_combined_file) and os.path.exists(train_label_file):
    print("Loading preprocessed training data...")
    X_train = np.load(train_combined_file)
    Y_train = np.load(train_label_file)
else:
    print("Processing training data (this may take time)...")
    X_train, Y_train = load_train_data()

    # Computes a mean and std for each channel (amplitude and phase) over all samples.
    global_mean = np.mean(X_train, axis=(0, 1, 2), keepdims=True)
    global_std = np.std(X_train, axis=(0, 1, 2), keepdims=True)

    # Normalize each channel separately
    X_train = (X_train - global_mean) / global_std

    # Save preprocessed arrays and normalization parameters for later use
    np.save(train_combined_file, X_train)
    np.save(train_label_file, Y_train)
    np.save(global_mean_file, global_mean)
    np.save(global_std_file, global_std)


# ----- Print Shapes -----
print(f"Training combined data shape: {X_train.shape}")
print(f"Training labels shape: {Y_train.shape}")



D. Preparing test data

In [ ]:
import os
import numpy as np
from glob import glob
from sklearn.utils import shuffle

# ----- Paths -----
segmented_dir = "/content/drive/MyDrive/my_dataset/testcwt_segmented/"
preprocessed_dir = "/content/drive/MyDrive/my_dataset/preprocessedcwt/"
os.makedirs(preprocessed_dir, exist_ok=True)

# Files to save preprocessed test data
test_combined_file = os.path.join(preprocessed_dir, "X_test_combinedcwt.npy")
test_label_file = os.path.join(preprocessed_dir, "Y_testcwt.npy")

# Global normalization parameters from training
global_mean_file = os.path.join(preprocessed_dir, "global_mean.npy")
global_std_file  = os.path.join(preprocessed_dir, "global_std.npy")

# ----- Function to load and combine test data -----
def load_test_data():
    # retrieve all segmented amplitude files for the test set
    amp_files = sorted(glob(os.path.join(segmented_dir, "segtestamp_*.npy")))
    X_combined = []  # store the combined amplitude-phase arrays
    Y = []           # Labels

    for amp_file in amp_files:
        # Derive corresponding phase file name by replacing "segtestamp_" with "segtestphase_"
        phase_file = amp_file.replace("segtestamp_", "segtestphase_")
        if not os.path.exists(phase_file):
            print(f"Skipping {amp_file}, corresponding phase file not found.")
            continue

        # Load amplitude and phase arrays
        amp_data = np.load(amp_file)      # Expected shape: (freq_bins, time_frames)
        phase_data = np.load(phase_file)  # Expected shape: (freq_bins, time_frames)

        print(f"Loaded amplitude file: {amp_file} with shape {amp_data.shape}")
        print(f"Loaded phase file: {phase_file} with shape {phase_data.shape}")

        if amp_data.shape != phase_data.shape:
            print(f"Mismatch in dimensions for {amp_file}, skipping.")
            continue

        # Stack amplitude and phase along a new last axis -> shape: (freq_bins, time_frames, 2)
        combined = np.stack([amp_data, phase_data], axis=-1)
        X_combined.append(combined)

        # Extract label from the file name.
        # Given "segtestamp_A1_P0_L1_T2_cwt_S1.npy",
        # we assume the activity label is encoded after "_A" (here, "1") and convert to 0-based indexing.
        try:
            label_str = amp_file.split("_A")[1].split("_")[0]
            label = int(label_str) - 1
        except Exception as e:
            print(f"Error extracting label from {amp_file}: {e}")
            continue

        Y.append(label)

    X_combined = np.array(X_combined)  # Shape: (num_samples, freq_bins, time_frames, 2)
    Y = np.array(Y)
    return X_combined, Y

# ----- Load or Process Test Data -----
if os.path.exists(test_combined_file) and os.path.exists(test_label_file):
    print("Loading preprocessed test data...")
    X_test = np.load(test_combined_file)
    Y_test = np.load(test_label_file)
else:
    print("Processing test data (this may take time)...")
    X_test, Y_test = load_test_data()

    # Ensure the global normalization parameters from training exist.
    if not (os.path.exists(global_mean_file) and os.path.exists(global_std_file)):
        raise ValueError("Global normalization parameters not found. Preprocess training data first.")

    # Load training normalization parameters
    global_mean = np.load(global_mean_file)
    global_std  = np.load(global_std_file)

    # Normalize test data using training parameters
    X_test = (X_test - global_mean) / global_std

    # Save preprocessed test data for future use
    np.save(test_combined_file, X_test)
    np.save(test_label_file, Y_test)

# ----- Print Shapes -----
print(f"Test combined data shape: {X_test.shape}")
print(f"Test labels shape: {Y_test.shape}")


E. Model architecture (CNN-LSTM)

In [ ]:
import os
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, BatchNormalization, Activation, Reshape, LSTM, Concatenate)
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split


# ========== 1) Load Preprocessed CWT Data ==========

preprocessed_dir = "/content/drive/MyDrive/my_dataset/preprocessedcwt/"  # Adjust as needed
X_train_file = os.path.join(preprocessed_dir, "X_train_combinedcwt.npy")
Y_train_file = os.path.join(preprocessed_dir, "Y_traincwt.npy")
X_test_file  = os.path.join(preprocessed_dir, "X_test_combinedcwt.npy")
Y_test_file  = os.path.join(preprocessed_dir, "Y_testcwt.npy")

X_train = np.load(X_train_file)
Y_train = np.load(Y_train_file)
X_test  = np.load(X_test_file)
Y_test  = np.load(Y_test_file)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# ========== 2) Shuffle Training Data ==========

X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

# ========== 4) Define the 2D CNN Model ==========

l2_lambda = 1e-5
input_shape = (75, 2000, 2)

input_data = Input(shape=input_shape, name="Input_Data")


# ========== Global Stream ==========
x1 = Conv2D(8, kernel_size=(4, 16), strides=(4, 8), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(input_data)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.3)(x1)

x1 = Conv2D(16, kernel_size=(8, 32), strides=(3, 6), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.3)(x1)

x1 = Conv2D(16, kernel_size=(4, 16), strides=(2, 4), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Reshape((-1, x1.shape[-2] * x1.shape[-1]))(x1)
x1 = (LSTM(128, return_sequences=False))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)


# ========== Local Stream ==========
x2 = Conv2D(8, kernel_size=(2, 2), strides=(1, 2), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(input_data)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.3)(x2)

x2 = Conv2D(16, kernel_size=(2, 2), strides=(1, 2), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.3)(x2)

x2 = Conv2D(16, kernel_size=(2, 2), strides=(1, 2), activation='relu',
            padding='same', kernel_regularizer=l2(l2_lambda))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.4)(x2)

x2 = Reshape((-1, x2.shape[-2] * x2.shape[-1]))(x2)
x2 = (LSTM(128, return_sequences=False))(x2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.4)(x2)


# ========== Merge Streams ==========
x = Concatenate()([x1, x2])

# ========== Dense Layers ==========
x = Dense(128, activation='relu', kernel_regularizer=l2(l2_lambda))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)


num_classes = 13
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_data, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

F. Training

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

# ----- Early Stopping -----
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


batch_size = 32
epochs = 200

history = model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, Y_val),
    callbacks=[early_stopping],
    verbose=1
)

# ----- Evaluate the Model on the Test Set -----
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# ----- Get Predictions on the Test Set -----
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# ----- Compute and Display the Confusion Matrix -----
conf_matrix = confusion_matrix(Y_test, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# ----- Display the Classification Report -----
print("Classification Report:")
print(classification_report(Y_test, predicted_labels))

G. Merging of static activities

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Group classes 2, 3, 4 into a single label "2" as the standing/sitting/lying falls under those classes respetively

def group_label(label):
    if label in [1, 2, 3]:
        return 2
    else:
        return label

y_true_grouped = [group_label(l) for l in Y_test]
y_pred_grouped = [group_label(l) for l in predicted_labels]

# Compute new accuracy
new_accuracy = accuracy_score(y_true_grouped, y_pred_grouped)
print("New Accuracy (with static activities merged):", new_accuracy)

conf_matrix_grouped = confusion_matrix(y_true_grouped, y_pred_grouped)
print("Grouped Confusion Matrix:\n", conf_matrix_grouped)

print("Grouped Classification Report:")
print(classification_report(y_true_grouped, y_pred_grouped))